# Sprint 3 - Interacting with data and the GitLab API

In this notebook, we will be accessing a test database populated with test data fields, isolating the email, checking if there is an existing GitLab ID associated with it, and making an API call to GitLab if no such ID is found.  We will then update our database with the ID.

----NOTE, need to write sql to reset database everytime we run for testing----

----if exists, drop table, etc----

### First make sure you have postgres running with the supplied database.  Schema still needs updated to match augur

Below is some code I copy+pasted out of my pgadmin GUI, but I built everything using the GUI, so I haven't tested this yet.  I was having difficulty trying to export my database, so I figured it would be best to just run this to build it for yourselves.

This can be run in your database manager of choice as SQL - To run in this notebook would require further use of the psycopg2 library, which would end up messier than necessary for this stage.

The SQL code below creates the "augurTest" databse that is owned by default by postgres.  In the python code below, you will need to enter credentials for whichever user is intended to read and make changes to the database.  This is the only thing that will need changed from machine to machine, and as long as you have the dependencies this shouldn run smoothly.  Feel free to add other email addresses!

CREATE DATABASE "augurTest"
    WITH 
    OWNER = postgres
    ENCODING = 'UTF8'
    LC_COLLATE = 'English_United States.1252'
    LC_CTYPE = 'English_United States.1252'
    TABLESPACE = pg_default
    CONNECTION LIMIT = -1;

ALTER ROLE postgres IN DATABASE "augurTest"
    SET search_path TO public;

CREATE TABLE public.gitlab
(
    cont_email character varying COLLATE pg_catalog."default" NOT NULL,
    gitlabid integer,
    gituser character varying COLLATE pg_catalog."default",
    CONSTRAINT gitlab_pkey PRIMARY KEY (cont_email)
);

TABLESPACE pg_default;

ALTER TABLE public.gitlab
    OWNER to postgres;
    
INSERT INTO gitlab (cont_email) VALUES ('brossn7@gmail.com'), ('briantheogre@gmail.com'), ('gms4kc@umsystem.edu');

Import necessary libraries.  psycopg2 handles python-database calls, requests and json handle the api calls.

In [1]:
import psycopg2
import requests
import json

First we need to connect to our database.  We create an instance called conn with the necessary details - this could be updated to read from a config file in the future.  Then psycopg2 requires we set up a cursor.  Then we execute a query to pull all the rows from the gitlab database.  This is a temporary way to pull sample data in, as we couldn't access the sample databases provided by Goggins, and this would end up pulling probably more data than we want in full production.

## THE CREDENTIALS BELOW APPLY TO MY LOCAL SYSTEM.  
## You will likely want to provide your own password, at the very least. 

In [9]:
conn = psycopg2.connect(host="localhost", port = 5432, database="augurTest", user="postgres", password="badpassword")
cur = conn.cursor()
cur.execute("SELECT contributors.cntrb_email, contributors.gitlab_username, contributors.gitlab_id FROM sean.contributors;")
tuples = cur.fetchall()

Next we set up a function APIcall to...you guessed it, make the api call.  The call still functions the way we showed in our prototype in sprint 2, however we've worked to avoid hardcoding anything here.  You'll notice a lot of print statements that aren't fully necessary but were great for testing.  The function accepts an email address pulled from our database and returns a list with the api-returned id # and gitlab username.

In [3]:
def APIcall(email):
    baseurl = 'https://gitlab.com/api/v4/users?search='
    url = ''.join([baseurl, email])
    req = requests.get(url, headers = {'private-token': 'zUpFd1JbsEYPdFL21kXK'})
    

    #print(req.json())
    #print("\n\n\n")
    j = req.json()
    if (len(j) == 0):
        #print("\nEmpty response!\n")
        data = (None, None)
        return data
    else:
        #print(email)
        #print(j[0]["name"])
        #print(j[0]["id"])
        #return j[0]["username"]
        data = (j[0]["id"], j[0]["username"])
        #return j[0]["id"]
        return data

print("Now updating table.  Depending on the size of your database, this may take a while.  Make a cup of tea.")
for i in tuples:
    x = i
    (email, user, labID) = x
    if (email == None):
        #print("\nEmail cannot be null!\n")
        pass
    elif (labID != None):
        #print("\nGitLabID found!\n")
        pass
    
    else:
        data = APIcall(email)
        ID = data[0]
        user = data[1]
        #sql = "UPDATE gitLab SET gitlabid = " + ID + " WHERE cont_email = %s"
        #email = "'" + email + "'"
        #cur.execute(sql, email)
        cur.execute("UPDATE sean.contributors SET gitlab_username = %s, gitlab_id = %s WHERE cntrb_email = %s", (user, ID, email))
print("Update complete!  Enjoy your updated table, now featuring gitlab identification!")

Now updating table.  Depending on the size of your database, this may take a while.  Make a cup of tea.


KeyboardInterrupt: 

After all that, we call conn.commit() to make our changes to the database, then we close our cursor and connection to the database.

In [ ]:
#One final test if database updated
#cur.execute("SELECT * FROM gitLab")
#tuples = cur.fetchall()
#print(tuples)

conn.commit()
cur.close()
conn.close()